Идея: 
1. сделать новый датасет из squad, где для каждого вопроса будет только три отрывка текста, среди которых может как быть правильный ответ, так и нет.
2. Сделать датасет, где кроме оригинального отрывка будет еще отрывок (вда отрывка) из другой темы

Получается, что у меня будет 6 датасетов и для них нужно сделать шесть моделей


In [52]:
import numpy as np
import pandas as pd

Загрузим наш датасет


In [53]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
   'squad_v2')

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/0e44b51f4035c15e218d53dc9eea5fe7123341982e524818b8500e4094fffb7b)


In [54]:
dataset.keys()

dict_keys(['train', 'validation'])

In [55]:
#train_data = dataset['train']
train_data = dataset['validation']


Переведем датасет в пандас

In [56]:
data = []
for item in train_data:
    if item['answers']['text']:
        data.append([item['title'], item['question'], item['context'], item['answers']['text'][0]])
    else:
        data.append([item['title'], item['question'], item['context'], float('nan')]) 

In [57]:
train_df = pd.DataFrame(data=data, columns=['title', 'question', 'context', 'answer'])

In [58]:
train_df

,title,question,context,answer
0,Normans,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,France
1,Normans,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,10th and 11th centuries
2,Normans,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"Denmark, Iceland and Norway"
3,Normans,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,Rollo
4,Normans,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,10th century
...,...,...,...,...
11868,Force,What is the seldom used force unit equal to on...,"The pound-force has a metric counterpart, less...",sthène
11869,Force,What does not have a metric counterpart?,"The pound-force has a metric counterpart, less...",NaN
11870,Force,What is the force exerted by standard gravity ...,"The pound-force has a metric counterpart, less...",NaN
11871,Force,What force leads to a commonly used unit of mass?,"The pound-force has a metric counterpart, less...",NaN


In [8]:
copy_train = train_df.copy()

Теперь его надо сгруппировать по 3 и по 2

Функция, которая соединяет последовательно три параграфа из Squad:

In [9]:
def create_merged_data(train_df, n=3):
    titles = train_df.title.unique()
    merged_data = []
    np.random.seed(1234)
    for title in titles:
        sub_df = train_df[train_df.title == title]
        contexts = sub_df.context.unique()
        l = len(contexts)
        for index, row in sub_df.iterrows():
            context_indx = np.where(contexts == row['context'])[0][0]
            #min index must be >=0 and <= len-n
            min_indx = min(max(0, context_indx - np.random.randint(0, n)), l-n)
            max_indx = min_indx + n
        
            texts = contexts[min_indx:max_indx]
            text_number = context_indx - min_indx + 1

            merged_data.append([row['question'], *texts, text_number, row['answer']])
    return merged_data

In [ ]:
merged_data_2 = create_merged_data(train_df, n=2)
merged_df_2 = pd.DataFrame(data= merged_data_2, columns=['question', 't1', 't2', 'text_number', 'answer'])
merged_df_2

,question,t1,t2,text_number,answer
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...",1,France
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...",1,10th and 11th centuries
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...",1,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...",1,Rollo
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...",1,10th century
...,...,...,...,...,...
11868,What is the seldom used force unit equal to on...,The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",2,sthène
11869,What does not have a metric counterpart?,The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",2,NaN
11870,What is the force exerted by standard gravity ...,The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",2,NaN
11871,What force leads to a commonly used unit of mass?,The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",2,NaN


In [ ]:
merged_data_3 = create_merged_data(train_df, n=3)
merged_df_3 = pd.DataFrame(data= merged_data_3, columns=['question', 't1', 't2', 't3', 'text_number', 'answer'])
merged_df_3

,question,t1,t2,t3,text_number,answer
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,France
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,10th and 11th centuries
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,Rollo
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,"The Norman dynasty had a major political, cult...","The English name ""Normans"" comes from the Fren...",1,10th century
...,...,...,...,...,...,...
11868,What is the seldom used force unit equal to on...,"For certain physical scenarios, it is impossib...",The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",3,sthène
11869,What does not have a metric counterpart?,"For certain physical scenarios, it is impossib...",The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",3,NaN
11870,What is the force exerted by standard gravity ...,"For certain physical scenarios, it is impossib...",The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",3,NaN
11871,What force leads to a commonly used unit of mass?,"For certain physical scenarios, it is impossib...",The connection between macroscopic nonconserva...,"The pound-force has a metric counterpart, less...",3,NaN


In [ ]:
path = '/content/drive/MyDrive/QA_training/merged datasets/squad'

In [ ]:
train_df.to_csv(path+'/initial_dev.csv', index=False)

In [ ]:
merged_df_2.to_csv(path+'/merged_by_2_dev.csv', index=False)


In [ ]:
merged_df_3.to_csv(path+'/merged_by_3_dev.csv', index=False)

Терь добавим к нашему параграфу еще один или два рандомных параграфа

In [59]:
def create_random_merged_data(df, n=3):
    titles = train_df.title.unique()
    merged_data = []
    np.random.seed(1234)
    for title in titles:
        sub_df = df[df.title == title]
        contexts = df[df.title != title].context.unique()
        for index, row in sub_df.iterrows():
            orig_text = row['context']
            random_texts = np.random.choice(contexts, size=n-1, replace=False)
            all_texts = np.concatenate(([orig_text], random_texts))
            np.random.shuffle(all_texts)
            true_text_id = np.where(all_texts == orig_text)[0][0] + 1
            merged_data.append([row['question'], *all_texts, true_text_id, row['answer']])
    return merged_data

In [60]:
%%time

merged_data_2 = create_random_merged_data(train_df, n=2)
merged_df_2 = pd.DataFrame(data= merged_data_2, columns=['question', 't1', 't2', 'text_number', 'answer'])
merged_df_2

CPU times: user 3.54 s, sys: 106 ms, total: 3.64 s
Wall time: 3.65 s


In [61]:
merged_df_2

,question,t1,t2,text_number,answer
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,"On the other hand, in the late 1980s the Weste...",1,France
1,When were the Normans in Normandy?,The immune system protects organisms from infe...,The Normans (Norman: Nourmands; French: Norman...,2,10th and 11th centuries
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,Newton's laws and Newtonian mechanics in gener...,1,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,The next major step occurred when James Watt d...,1,Rollo
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,Guo Shoujing applied mathematics to the constr...,1,10th century
...,...,...,...,...,...
11868,What is the seldom used force unit equal to on...,"The pound-force has a metric counterpart, less...","Reactive oxygen species, such as superoxide io...",1,sthène
11869,What does not have a metric counterpart?,"The pound-force has a metric counterpart, less...",The economy of Victoria is highly diversified:...,1,NaN
11870,What is the force exerted by standard gravity ...,"The pound-force has a metric counterpart, less...",The executive summary of the WG I Summary for ...,1,NaN
11871,What force leads to a commonly used unit of mass?,"In 2007, BSkyB and Virgin Media became involve...","The pound-force has a metric counterpart, less...",2,NaN


In [62]:
merged_data_3 = create_random_merged_data(train_df, n=3)
merged_df_3 = pd.DataFrame(data= merged_data_3, columns=['question', 't1', 't2', 't3', 'text_number', 'answer'])
merged_df_3

,question,t1,t2,t3,text_number,answer
0,In what country is Normandy located?,Sky UK Limited (formerly British Sky Broadcast...,The Normans (Norman: Nourmands; French: Norman...,"On the other hand, in the late 1980s the Weste...",2,France
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,The other third of the water flows through the...,"From the late 1340s onwards, people in the cou...",1,10th and 11th centuries
2,From which countries did the Norse originate?,Newton's First Law of Motion states that objec...,The Normans (Norman: Nourmands; French: Norman...,Newton's laws and Newtonian mechanics in gener...,2,"Denmark, Iceland and Norway"
3,Who was the Norse leader?,The Rankine cycle is the fundamental thermodyn...,The Normans (Norman: Nourmands; French: Norman...,The next major step occurred when James Watt d...,2,Rollo
4,What century did the Normans first gain their ...,One of the first known experiments on the rela...,"From the Eocene onwards, the ongoing Alpine or...",The Normans (Norman: Nourmands; French: Norman...,3,10th century
...,...,...,...,...,...,...
11868,What is the seldom used force unit equal to on...,"The pound-force has a metric counterpart, less...",Some civil disobedients feel it is incumbent u...,The war was fought primarily along the frontie...,1,sthène
11869,What does not have a metric counterpart?,The steam engine contributed much to the devel...,Studies on income inequality and growth have s...,"The pound-force has a metric counterpart, less...",3,NaN
11870,What is the force exerted by standard gravity ...,"In Japan, at the end of the Asuka period (538–...","For the next three hundred years, Scotland was...","The pound-force has a metric counterpart, less...",3,NaN
11871,What force leads to a commonly used unit of mass?,"The pound-force has a metric counterpart, less...",Like much of the south Atlantic region of the ...,"For the Conservatives, the main disappointment...",1,NaN


In [63]:
path = '/content/drive/MyDrive/QA_training/merged datasets/squad'


In [14]:
merged_df_2.to_csv(path+'/random_merged_by_2_dev.csv', index=False)

In [15]:
merged_df_3.to_csv(path+'/random_merged_by_3_dev.csv', index=False)